In [1]:
import threading
import numpy as np
import time
import subprocess
import os
import psutil
import tensorflow as tf
import tempfile
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import LambdaCallback

2024-08-08 12:55:52.017932: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import threading
import time
import subprocess
import os
import psutil
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import LambdaCallback
import tempfile
import re
import tf2onnx
import coremltools as ct
import onnxruntime as ort
import coremltools as ct
from coremltools.models import MLModel

scikit-learn version 1.3.2 is not supported. Minimum required version: 0.17. Maximum required version: 1.1.2. Disabling scikit-learn conversion API.
XGBoost version 2.1.0 has not been tested with coremltools. You may run into unexpected errors. XGBoost 1.4.2 is the most recent version that has been tested.
Torch version 2.2.2 has not been tested with coremltools. You may run into unexpected errors. Torch 2.2.0 is the most recent version that has been tested.
/opt/anaconda3/envs/env_name/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 全局变量用于存储子线程的输出
thread_output = {}

# 加载 IMDB 数据集
max_features = 20000  # 使用的单词数量
maxlen = 100  # 每条评论的最大长度

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

# 数据预处理和填充
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

In [ ]:


# 构建 LSTM 模型
model = Sequential([
    Embedding(input_dim=max_features, output_dim=128, input_length=maxlen),
    LSTM(units=128, return_sequences=True),  # 第一层LSTM，返回序列以叠加另一层LSTM
    LSTM(units=128),  # 第二层LSTM
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # 二分类问题，使用sigmoid激活函数
])

# 编译模型
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 自定义批次生成器
def batch_generator(X, y, batch_size):
    while True:
        for start in range(0, len(X), batch_size):
            end = min(start + batch_size, len(X))
            yield X[start:end], y[start:end]

batch_size = 128
train_gen = batch_generator(X_train, y_train, batch_size)

# 监控资源使用情况
def monitor_resources():
    memory_info = psutil.virtual_memory()
    print(f"Memory usage: {memory_info.percent}%")
    cpu_usage = psutil.cpu_percent(interval=1)
    print(f"CPU usage: {cpu_usage}%")

# 训练模型并监控资源使用情况
steps_per_epoch = len(X_train) // batch_size
model.fit(train_gen, epochs=2, steps_per_epoch=steps_per_epoch, verbose=2, 
          callbacks=[LambdaCallback(on_epoch_end=lambda epoch, logs: monitor_resources())])

# 评估模型
score = model.evaluate(X_test, y_test, verbose=0)
print(f'Test accuracy: {score[1]}')

model.save('./models_train/imdb_lstm_model.h5')

Epoch 1/2
Memory usage: 90.4%
CPU usage: 9.8%
195/195 - 42s - loss: 0.4247 - accuracy: 0.7954 - 42s/epoch - 215ms/step
Epoch 2/2
Memory usage: 90.2%
CPU usage: 7.3%
195/195 - 39s - loss: 0.2283 - accuracy: 0.9107 - 39s/epoch - 200ms/step
Test accuracy: 0.8289999961853027


### IMDB-LSTM-H5

### H5

In [14]:
import threading
import time
import subprocess
import tempfile
import os
import re
import psutil
import numpy as np
import tensorflow as tf

# 初始化全局变量
thread_output = {}
duration = 0
inference_duration = 0

# 定义监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 定义保存模型的函数
def save_model(stop_event, model):
    global duration
    global inference_duration
    start_time = time.time()

    for i in range(0, 1):
        model.save("imdb_lstm_model.h5")

    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save TensorFlow SavedModel: {duration} seconds')
    
    # 测量推理时间
    X_test_sample = X_test[:10000]  # 选择前1000个样本进行推理
    batch_size = 128
    num_batches = (len(X_test_sample) + batch_size - 1) // batch_size

    start_time_inference = time.time()
    for _ in range(1):  # 推理循环次数为1
        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = min(batch_start + batch_size, len(X_test_sample))
            batch = X_test_sample[batch_start:batch_end]
            model.predict(batch)
    end_time_inference = time.time()

    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {len(X_test_sample)} samples: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()
                    process.wait()
        
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

# 创建和启动线程
stop_event = threading.Event()
model = tf.keras.models.load_model("./models_train/imdb_lstm_model.h5")  # 重新加载模型以进行保存性能监控

# # 加载IMDB数据集进行处理
# max_features = 20000
# maxlen = 100
# (X_train, y_train), (X_test, y_test) = tf.keras.datasets.imdb.load_data(num_words=max_features)
# X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=maxlen)

thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread1.start()
thread2.start()
thread3.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "CPU consume" 和 "GPU Power" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or line.startswith('CPU Power:')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

# 保存过滤后的内容到文件
output_file_name = './imbd_models/output_lstm/output-imdb-lstm-h5.txt'
with open(output_file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {output_file_name}")

# 提取每一个采样点的数字，即CPU和GPU的具体mV
numbers = []
for line in filtered_lines:
    match = re.search(r'[\d.]+', line)
    if match:
        numbers.append(float(match.group()))

delta_time = duration * 2 / len(filtered_lines)
numbers_scaled = [num * delta_time for num in numbers]
total_energy_consumption = sum(numbers_scaled)
print(f"Total energy consumption: {total_energy_consumption:.2f} mV")


Subprocess started.
Time taken to save TensorFlow SavedModel: 0.20819616317749023 seconds
1/1 [==============================] - 1s 951ms/step
Time taken for inference on 10000 samples: 30.1386 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./imbd_models/output_lstm/output-imdb-lstm-h5.txt
Total energy consumption: 2128.41 mV


### onnx

In [17]:
import threading
import time
import subprocess
import tempfile
import os
import re
import psutil
import numpy as np
import tensorflow as tf
import tf2onnx
import onnxruntime as ort
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 初始化全局变量
thread_output = {}
duration = 0
inference_duration = 0

# 定义监控资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 定义保存模型的函数
def save_model(stop_event, model):
    global duration
    global inference_duration
    global X_test  # 确保引用全局变量

    start_time = time.time()
    
    # 使用 tf2onnx 保存模型为ONNX格式
    spec = (tf.TensorSpec((None, 100), tf.float32, name="input"),)
    output_path = "imdb_lstm_model.onnx"
    
    for i in range(1):
        model_proto, _ = tf2onnx.convert.from_keras(model, input_signature=spec, output_path=output_path)
    
    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save ONNX model: {duration} seconds')
    
    # 测量推理时间
    # 使用 ONNX Runtime 加载模型
    ort_session = ort.InferenceSession(output_path)
    
    # 准备 ONNX 输入
    X_test_sample = X_test[:10000].astype(np.float32)  # 选择前1000个样本进行推理
    
    # 测量推理时间
    start_time_inference = time.time()
    
    # 获取模型输入名称
    input_name = ort_session.get_inputs()[0].name
    
    # 批量推理部分
    batch_size = 128
    num_samples = X_test_sample.shape[0]
    num_batches = (num_samples + batch_size - 1) // batch_size

    for _ in range(1):  # 推理循环次数为 1
        for batch_idx in range(num_batches):
            batch_start = batch_idx * batch_size
            batch_end = min(batch_start + batch_size, num_samples)
            batch = X_test_sample[batch_start:batch_end]
            ort_outs = ort_session.run(None, {input_name: batch})
    
    end_time_inference = time.time()
    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {len(X_test_sample)} samples: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()
                    process.wait()
        
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

# 创建和启动线程
stop_event = threading.Event()
model = tf.keras.models.load_model("./models_train/imdb_lstm_model.h5")  # 重新加载模型以进行保存性能监控

# 确保 X_test 被加载以便进行推理
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=20000)
X_test = pad_sequences(X_test, maxlen=100)

thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread1.start()
thread2.start()
thread3.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "GPU Power" 和 "CPU Power" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or 
                  line.startswith('CPU Power:')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

# 保存过滤后的内容到文件
output_file_name = './imbd_models/output_lstm/output-imdb-lstm-onnx.txt'
with open(output_file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {output_file_name}")

# 提取每一个采样点的数字，即CPU和GPU的具体mV
numbers = []
for line in filtered_lines:
    match = re.search(r'[\d.]+', line)
    if match:
        numbers.append(float(match.group())) 

delta_time = duration * 2 / len(filtered_lines) if len(filtered_lines) > 0 else 0
numbers_scaled = [num * delta_time for num in numbers]
total_energy_consumption = sum(numbers_scaled)
print(f"Total energy consumption: {total_energy_consumption:.2f} mV")

Subprocess started.
Time taken to save ONNX model: 2.283402919769287 seconds
Time taken for inference on 10000 samples: 4.3970 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./imbd_models/output_lstm/output-imdb-lstm-onnx.txt
Total energy consumption: 35187.12 mV


### CORE ML

In [ ]:
# 初始化全局变量
thread_output = {}
duration = 0
inference_duration = 0
X_test = None  # 确保在测试时能够使用到

# 定义监控资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 定义保存模型的函数
def save_model(stop_event, model):
    global duration
    global inference_duration
    global X_test  # 确保引用全局变量

    start_time = time.time()
    
    # 将TensorFlow Keras模型转换为Core ML模型
    for i in range(1):
        try:
            # 使用实际的输入层名称和适当的形状
            input_feature = ct.TensorType(name="embedding_input", shape=(1, 100))
            mlmodel = ct.convert(model, inputs=[input_feature])
            mlmodel.save("imdb_lstm_model.mlpackage")
        except Exception as e:
            print(f"Error saving model: {e}")
    
    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save Core ML model: {duration} seconds')
    
    # 测量推理时间
    # 使用 Core ML 加载模型
    coreml_model = MLModel("imdb_lstm_model.mlpackage")
    
    # 准备 Core ML 输入
    X_test_sample = X_test[0].astype(np.float32)  # 选择第一个样本进行推理

    # 确保形状正确
    X_test_sample = X_test_sample.reshape((1, 100))

    # 测量推理时间
    start_time_inference = time.time()

    for i in range(1):
        input_data = {"embedding_input": X_test_sample}
        coreml_out = coreml_model.predict(input_data)

    end_time_inference = time.time()
    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on 1 sample, repeated 32 times: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()
                    process.wait()
        
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

# 创建和启动线程
stop_event = threading.Event()
model = tf.keras.models.load_model("./models_train/imdb_lstm_model.h5") 

# 确保 X_test 被加载以便进行推理
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=20000)
X_test = pad_sequences(X_test, maxlen=100)

thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread1.start()
thread2.start()
thread3.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "GPU Power" 和 "CPU Power" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or 
                  line.startswith('CPU Power:')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

# 保存过滤后的内容到文件
output_file_name = './imbd_models/output_lstm/output-imdb-lstm-coreml.txt'
with open(output_file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {output_file_name}")

# 提取每一个采样点的数字，即CPU和GPU的具体mV
numbers = []
for line in filtered_lines:
    match = re.search(r'[\d.]+', line)
    if match:
        numbers.append(float(match.group())) 

delta_time = duration * 2 / len(filtered_lines) if len(filtered_lines) > 0 else 0
numbers_scaled = [num * delta_time for num in numbers]
total_energy_consumption = sum(numbers_scaled)
print(f"Total energy consumption: {total_energy_consumption:.2f} mV")


Subprocess started.


When both 'convert_to' and 'minimum_deployment_target' not specified, 'convert_to' is set to "mlprogram" and 'minimum_deployment_target' is set to ct.target.iOS15 (which is same as ct.target.macOS12). Note: the model will not run on systems older than iOS15/macOS12/watchOS8/tvOS15. In order to make your model run on older system, please set the 'minimum_deployment_target' to iOS14/iOS13. Details please see the link: https://apple.github.io/coremltools/docs-guides/source/target-conversion-formats.html
Converting TF Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 49974.69 ops/s]
Input ls elem type unknown. Override with <class 'coremltools.converters.mil.mil.types.type_tensor.tensor.<locals>.tensor'>
Converting TF Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 31587.01 ops/s]
Input ls elem type unknown. Override with <class 'coremltools.converters.mil.mil.types.type_tensor.tensor.<locals>.tensor'>
Converting TF Frontend ==> MIL Ops: 100%|██████████| 40/40 [00:00<00

Time taken to save Core ML model: 5.234626054763794 seconds
Time taken for inference on 1 sample, repeated 32 times: 6.3541 seconds
Resource monitoring finished.
Subprocess finished.
Content saved to ./imbd_models/output_lstm/output-imdb-lstm-coreml.txt
Total energy consumption: 19735.98 mV


In [18]:
import coremltools as ct
import psutil
import subprocess
import threading
import time
import tempfile
import os
import re
import numpy as np
import tensorflow as tf
from coremltools.models import MLModel
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.callbacks import LambdaCallback

# 初始化全局变量
thread_output = {}
duration = 0
inference_duration = 0

# 定义监控资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# # 定义保存模型的函数
# def save_model(stop_event, model):
#     global duration
#     global inference_duration
#     global X_test  # 确保引用全局变量

#     start_time = time.time()
    
#     # 将TensorFlow Keras模型转换为Core ML模型
#     try:
#         # 使用实际的输入层名称和适当的形状
#         input_feature = ct.TensorType(name="embedding_input", shape=(1, 100))
#         mlmodel = ct.convert(model, inputs=[input_feature])
#         mlmodel.save("imdb_lstm_model.mlpackage")
#     except Exception as e:
#         print(f"Error saving model: {e}")
    
#     end_time = time.time()
#     duration = end_time - start_time
#     print(f'Time taken to save Core ML model: {duration} seconds')
    
#     # 测量推理时间
#     # 使用 Core ML 加载模型
#     coreml_model = MLModel("imdb_lstm_model.mlpackage")
    
#     # 准备 Core ML 输入
#     X_test_sample = X_test[:1000].astype(np.float32)  # 选择前1000个样本进行推理

#     # 批量推理部分，设置 batch size 为 32
#     batch_size = 1
#     num_samples = X_test_sample.shape[0]
#     num_batches = (num_samples + batch_size - 1) // batch_size

#     start_time_inference = time.time()

#     # 执行批量推理
#     for batch_idx in range(num_batches):
#         batch_start = batch_idx * batch_size
#         batch_end = min(batch_start + batch_size, num_samples)
#         batch = X_test_sample[batch_start:batch_end].reshape((batch_end - batch_start, 100))
#         input_data = {"embedding_input": batch}
#         coreml_out = coreml_model.predict(input_data)

#     end_time_inference = time.time()
#     inference_duration = end_time_inference - start_time_inference
#     print(f'Time taken for inference on {num_samples} samples: {inference_duration:.4f} seconds')

#     stop_event.set()  # 触发停止其他线程

def save_model(stop_event, model):
    global duration
    global inference_duration
    global X_test  # 确保引用全局变量

    start_time = time.time()
    
    # 将 TensorFlow Keras 模型转换为 Core ML 模型时，确保输入尺寸正确
    try:
        # 如果模型确实需要处理变长输入，可以尝试指定具体的批量大小
        input_feature = ct.TensorType(name="embedding_input", shape=(1, 100))
        mlmodel = ct.convert(model, inputs=[input_feature], source='tensorflow')
        mlmodel.save("imdb_lstm_model.mlpackage")
        print("Model saved successfully.")
    except Exception as e:
        print(f"Error saving model: {e}")


    end_time = time.time()
    duration = end_time - start_time
    print(f"Time taken to save Core ML model: {duration:.2f} seconds")

    
    # 测量推理时间
    # 使用 Core ML 加载模型
    coreml_model = MLModel("imdb_lstm_model.mlpackage")
    
    # 准备 Core ML 输入，这里假设X_test已经被适当地预处理为序列
    X_test_sample = X_test[:1000].astype(np.float32)  # 选择前1000个样本进行推理

    # 批量推理部分，设置 batch size 为 32
    batch_size = 128
    num_samples = X_test_sample.shape[0]
    num_batches = (num_samples + batch_size - 1) // batch_size

    start_time_inference = time.time()

    # 执行批量推理
    predictions = []
    for batch_idx in range(num_batches):
        batch_start = batch_idx * batch_size
        batch_end = min(batch_start + batch_size, num_samples)
        batch = X_test_sample[batch_start:batch_end]
        for sample in batch:  # 处理每个样本，因为模型只接受单个样本
            input_data = {"embedding_input": sample.reshape(1, 100)}  # 重塑每个样本为 (1, 100)
            try:
                batch_predictions = coreml_model.predict(input_data)
                predictions.append(batch_predictions)
            except Exception as e:
                print(f"Error during batch inference {batch_idx}: {e}")


    end_time_inference = time.time()
    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {num_samples} samples: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程


# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()
                    process.wait()
        
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))

# 创建和启动线程
stop_event = threading.Event()
model = tf.keras.models.load_model("./models_train/imdb_lstm_model.h5")

# 确保 X_test 被加载以便进行推理
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=20000)
X_test = pad_sequences(X_test, maxlen=100)

thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))
thread1.start()
thread2.start()
thread3.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
# 将内容按行拆分
lines = content.split('\n')
# 筛选出以 "GPU Power" 和 "CPU Power" 开头的行
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or 
                  line.startswith('CPU Power:')]
# 将筛选后的行合并为一个字符串，每行之间用换行符分隔
filtered_content = '\n'.join(filtered_lines)

# 保存过滤后的内容到文件
output_file_name = './imbd_models/output_lstm/output-imdb-lstm-coreml.txt'
with open(output_file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {output_file_name}")

# 提取每一个采样点的数字，即CPU和GPU的具体mV
numbers = []
for line in filtered_lines:
    match = re.search(r'[\d.]+', line)
    if match:
        numbers.append(float(match.group())) 

delta_time = duration * 2 / len(filtered_lines) if len(filtered_lines) > 0 else 0
numbers_scaled = [num * delta_time for num in numbers]
total_energy_consumption = sum(numbers_scaled)
print(f"Total energy consumption: {total_energy_consumption:.2f} mV")


When both 'convert_to' and 'minimum_deployment_target' not specified, 'convert_to' is set to "mlprogram" and 'minimum_deployment_target' is set to ct.target.iOS15 (which is same as ct.target.macOS12). Note: the model will not run on systems older than iOS15/macOS12/watchOS8/tvOS15. In order to make your model run on older system, please set the 'minimum_deployment_target' to iOS14/iOS13. Details please see the link: https://apple.github.io/coremltools/docs-guides/source/target-conversion-formats.html


Subprocess started.


Converting TF Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 45484.32 ops/s]
Input ls elem type unknown. Override with <class 'coremltools.converters.mil.mil.types.type_tensor.tensor.<locals>.tensor'>
Converting TF Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 54878.74 ops/s]
Input ls elem type unknown. Override with <class 'coremltools.converters.mil.mil.types.type_tensor.tensor.<locals>.tensor'>
Converting TF Frontend ==> MIL Ops: 100%|██████████| 40/40 [00:00<00:00, 11107.80 ops/s]
Saving value type of int64 into a builtin type of int32, might lose precision!
Saving value type of int64 into a builtin type of int32, might lose precision!
Converting TF Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 67339.74 ops/s]
Input ls elem type unknown. Override with <class 'coremltools.converters.mil.mil.types.type_tensor.tensor.<locals>.tensor'>
Converting TF Frontend ==> MIL Ops: 100%|██████████| 14/14 [00:00<00:00, 33573.62 ops/s]
Input ls elem type unknow

Model saved successfully.
Time taken to save Core ML model: 4.58 seconds
Time taken for inference on 1000 samples: 205.5911 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./imbd_models/output_lstm/output-imdb-lstm-coreml.txt
Total energy consumption: 19788.38 mV


### lstm-pytorch

In [19]:
# 加载 IMDB 数据集 （确保不重新训练模型也能推理）
max_features = 20000  # 使用的单词数量
maxlen = 100  # 每条评论的最大长度

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

# 数据预处理和填充
X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

In [20]:
import tensorflow as tf
import torch
import torch.nn as nn
import torch
import h5py

# 用pytorch定义LSTM模型

class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm1 = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.lstm2 = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)
        self.fc1 = nn.Linear(hidden_dim, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm1(x)
        x, (h_n, c_n) = self.lstm2(x)  # 仅获取最后一个时间步的输出
        x = h_n[-1]  # 取最后一个层的隐藏状态
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

def save_keras_lstm_to_pytorch(modelpath):
    # 加载已经训练的TensorFlow模型，保存参数
    model = tf.keras.models.load_model(modelpath)
    model.save_weights('imdb_lstm_weights.h5')

    # 加载 Keras 权重
    # 打开 HDF5 文件
    weights = h5py.File('imdb_lstm_weights.h5', 'r')

    # 创建 PyTorch 模型实例
    pytorch_model = LSTMModel(max_features, 128, 128)

    # 加载 Embedding 层权重
    pytorch_model.embedding.weight.data.copy_(torch.from_numpy(weights['embedding/embedding/embeddings:0'][()]))

    # 加载第一个 LSTM 层权重
    # LSTM层包括kernel（即weights_ih）和recurrent_kernel（即weights_hh）以及bias
    pytorch_model.lstm1.weight_ih_l0.data.copy_(torch.from_numpy(weights['lstm/lstm/lstm_cell/kernel:0'][()].transpose()))
    pytorch_model.lstm1.weight_hh_l0.data.copy_(torch.from_numpy(weights['lstm/lstm/lstm_cell/recurrent_kernel:0'][()].transpose()))
    pytorch_model.lstm1.bias_ih_l0.data.copy_(torch.from_numpy(weights['lstm/lstm/lstm_cell/bias:0'][()]))
    pytorch_model.lstm1.bias_hh_l0.data.fill_(0)  # 清零，因为Keras中的偏置是合并的

    # 加载第二个 LSTM 层权重
    pytorch_model.lstm2.weight_ih_l0.data.copy_(torch.from_numpy(weights['lstm_1/lstm_1/lstm_cell_1/kernel:0'][()].transpose()))
    pytorch_model.lstm2.weight_hh_l0.data.copy_(torch.from_numpy(weights['lstm_1/lstm_1/lstm_cell_1/recurrent_kernel:0'][()].transpose()))
    pytorch_model.lstm2.bias_ih_l0.data.copy_(torch.from_numpy(weights['lstm_1/lstm_1/lstm_cell_1/bias:0'][()]))
    pytorch_model.lstm2.bias_hh_l0.data.fill_(0)  # 清零

    # 加载第一个 Dense 层权重和偏置
    pytorch_model.fc1.weight.data.copy_(torch.from_numpy(weights['dense/dense/kernel:0'][()].transpose()))
    pytorch_model.fc1.bias.data.copy_(torch.from_numpy(weights['dense/dense/bias:0'][()]))

    # 加载第二个 Dense 层权重和偏置
    pytorch_model.fc2.weight.data.copy_(torch.from_numpy(weights['dense_1/dense_1/kernel:0'][()].transpose()))
    pytorch_model.fc2.bias.data.copy_(torch.from_numpy(weights['dense_1/dense_1/bias:0'][()]))

    weights.close()  # 关闭文件

    # 现在你的 PyTorch 模型已经加载了 Keras 模型的所有权重

    return pytorch_model

    # 将模型设置为评估模式

# pytorch_model = save_keras_lstm_to_pytorch("./models_train/imdb_lstm_model.h5")

In [ ]:
thread_output = {}
duration = 0
inference_duration = 0  # 添加用于存储推理时间的变量

# 定义监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

# 定义保存模型的函数
def save_model(stop_event, model):
    global duration
    global inference_duration
    print("Saving model...")
    start_time = time.time()

    # 保存整个模型为 PyTorch 格式
    pytorch_model = save_keras_lstm_to_pytorch("./models_train/imdb_lstm_model.h5")
    for i in range(1):
        torch.save(pytorch_model, 'imdb_lstm_model.pth')

    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save PyTorch model: {duration:.2f} seconds')

    
    # 准备输入数据
    X_test_sample = X_test[:10].astype(np.float32)  # 选择前10个样本进行推理
    X_test_sample_torch = torch.from_numpy(X_test_sample).long()  # 转换为整数张量

    # 测量推理时间
    start_time_inference = time.time()
    # 加载模型
    pytorch_model.eval()

    # 假设 X_test 是你要用于推理的输入数据
    for i in range(1):
        with torch.no_grad():
            output = pytorch_model(X_test_sample_torch)
            print("Inference output:", output)

    end_time_inference = time.time()
    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {len(X_test_sample)} samples: {inference_duration:.4f} seconds')


    stop_event.set()  # 触发停止其他线程

# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        # 使用临时文件来存储子进程输出
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                # 尝试终止进程
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()  # 如果超时，则强制终止
                    process.wait()
        
        # 读取临时文件的内容
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))


stop_event = threading.Event()
model = tf.keras.models.load_model("./models_train/imdb_lstm_model.h5") 

# 创建和启动线程
thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))


thread1.start()
thread2.start()
thread3.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
lines = content.split('\n')
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or 
                  line.startswith('CPU Power:')]

filtered_content = '\n'.join(filtered_lines)

output_file_name = './imbd_models/output_lstm/output-imdb-lstm-pth.txt'
with open(output_file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {output_file_name}")

filtered_lines_count = len(filtered_lines)
#print(filtered_lines_count)
#print(filtered_lines)

duration = inference_duration
# 确保 filtered_lines_count 不为零
if filtered_lines_count > 0:
    # 提取每一个采样点的数字，即CPU和GPU的具体mV
    numbers = []
    for line in filtered_lines:
        match = re.search(r'[\d.]+', line)
        if match:
            numbers.append(float(match.group()))

    delta_time = duration * 2 / filtered_lines_count
    numbers_scaled = [num * delta_time for num in numbers]
    total_energy_consumption = sum(numbers_scaled)
    print(f"Total energy consumption: {total_energy_consumption:.2f} mV")
else:
    print("No filtered lines to process.")

Saving model...
Subprocess started.
Time taken to save PyTorch model: 3.17 seconds
Inference output: tensor([[0.0524],
        [0.9960],
        [0.9824],
        [0.1588],
        [0.9983],
        [0.9150],
        [0.9923],
        [0.0162],
        [0.7153],
        [0.9792]])
Inference output: tensor([[0.0524],
        [0.9960],
        [0.9824],
        [0.1588],
        [0.9983],
        [0.9150],
        [0.9923],
        [0.0162],
        [0.7153],
        [0.9792]])
Inference output: tensor([[0.0524],
        [0.9960],
        [0.9824],
        [0.1588],
        [0.9983],
        [0.9150],
        [0.9923],
        [0.0162],
        [0.7153],
        [0.9792]])
Inference output: tensor([[0.0524],
        [0.9960],
        [0.9824],
        [0.1588],
        [0.9983],
        [0.9150],
        [0.9923],
        [0.0162],
        [0.7153],
        [0.9792]])
Inference output: tensor([[0.0524],
        [0.9960],
        [0.9824],
        [0.1588],
        [0.9983],
        [0.91

Subprocess finished.Subprocess finished.

Subprocess finished.


In [ ]:
thread_output = {}
duration = 0
inference_duration = 0  # 添加用于存储推理时间的变量

# 定义监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

def save_model(stop_event, model):
    global duration
    global inference_duration
    global X_test  # 确保引用全局变量

    print("Saving model...")
    start_time = time.time()

    # 假设此函数正确将Keras模型转换为PyTorch模型
    pytorch_model = save_keras_lstm_to_pytorch("./models_train/imdb_lstm_model.h5")
    torch.save(pytorch_model, 'imdb_lstm_model.pth')

    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save PyTorch model: {duration:.2f} seconds')

    # 准备输入数据
    X_test_sample = X_test[:1000].astype(np.float32)  # 选择前10个样本进行推理
    X_test_sample_torch = torch.from_numpy(X_test_sample).long()  # 转换为整数张量

    # 测量推理时间
    start_time_inference = time.time()
    pytorch_model.eval()  # 设置模型为评估模式

    # 推理，批处理逻辑
    batch_size = 1  # 设置模拟的批大小为128
    predictions = []
    # 通过多次执行推理来模拟大批量处理
    for _ in range(batch_size // len(X_test_sample)):  # 这里使用实际样本数量来计算需要模拟批处理的次数
        with torch.no_grad():
            output = pytorch_model(X_test_sample_torch)
            predictions.append(output)
            print("Inference output:", output)

    end_time_inference = time.time()
    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {len(X_test_sample)} samples: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程



# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        # 使用临时文件来存储子进程输出
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                # 尝试终止进程
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()  # 如果超时，则强制终止
                    process.wait()
        
        # 读取临时文件的内容
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))


stop_event = threading.Event()
model = tf.keras.models.load_model("./models_train/imdb_lstm_model.h5") 

# 创建和启动线程
thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))


thread1.start()
thread2.start()
thread3.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
lines = content.split('\n')
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or 
                  line.startswith('CPU Power:')]

filtered_content = '\n'.join(filtered_lines)

output_file_name = './imbd_models/output_lstm/output-imdb-lstm-pth.txt'
with open(output_file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {output_file_name}")

filtered_lines_count = len(filtered_lines)

duration = inference_duration
# 确保 filtered_lines_count 不为零
if filtered_lines_count > 0:
    # 提取每一个采样点的数字，即CPU和GPU的具体mV
    numbers = []
    for line in filtered_lines:
        match = re.search(r'[\d.]+', line)
        if match:
            numbers.append(float(match.group()))

    delta_time = duration * 2 / filtered_lines_count
    numbers_scaled = [num * delta_time for num in numbers]
    total_energy_consumption = sum(numbers_scaled)
    print(f"Total energy consumption: {total_energy_consumption:.2f} mV")
else:
    print("No filtered lines to process.")

Saving model...
Subprocess started.
Time taken to save PyTorch model: 2.04 seconds
Time taken for inference on 1000 samples: 0.0003 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./imbd_models/output_lstm/output-imdb-lstm-pth.txt
Total energy consumption: 1.84 mV


In [21]:
thread_output = {}
duration = 0
inference_duration = 0  # 添加用于存储推理时间的变量

# 定义监控保存模型时的资源使用率的线程函数
def monitor_resources_during_save(stop_event):
    cpu_usage = []
    gpu_usage = []

    while not stop_event.is_set():
        cpu_usage.append(psutil.cpu_percent(interval=0.1))
        try:
            gpu_output = subprocess.check_output(['nvidia-smi', 
                                                  '--query-gpu=utilization.gpu', '--format=csv,noheader,nounits'])
            gpu_usage.append(int(gpu_output.strip()))
        except Exception as e:
            gpu_usage.append(None)  # 如果没有GPU或nvidia-smi命令失败，则记录None

    # 保存监测结果
    thread_output['cpu_usage'] = cpu_usage
    thread_output['gpu_usage'] = gpu_usage
    print("Resource monitoring finished.")

def save_model(stop_event, model):
    global duration
    global inference_duration
    global X_test  # 确保引用全局变量

    print("Saving model...")
    start_time = time.time()

    # 假设此函数正确将Keras模型转换为PyTorch模型
    pytorch_model = save_keras_lstm_to_pytorch("./models_train/imdb_lstm_model.h5")
    torch.save(pytorch_model, 'imdb_lstm_model.pth')

    end_time = time.time()
    duration = end_time - start_time
    print(f'Time taken to save PyTorch model: {duration:.2f} seconds')

    # 准备输入数据
    X_test_sample = X_test[:10000]  # 选择前1000个样本进行推理
    X_test_sample_torch = torch.from_numpy(X_test_sample).long()  # 正确转换为整数张量

    # 测量推理时间
    start_time_inference = time.time()
    pytorch_model.eval()  # 设置模型为评估模式

    # 推理，批处理逻辑
    batch_size = 128  # 实际设置批量大小为128
    predictions = []
    for i in range(0, len(X_test_sample_torch), batch_size):
        batch = X_test_sample_torch[i:i + batch_size]
        with torch.no_grad():
            output = pytorch_model(batch)
            predictions.append(output)
            #print("Inference output:", output)

    end_time_inference = time.time()
    inference_duration = end_time_inference - start_time_inference
    print(f'Time taken for inference on {len(X_test_sample)} samples: {inference_duration:.4f} seconds')

    stop_event.set()  # 触发停止其他线程


# 运行外部脚本并捕获输出
def run_script(stop_event):
    try:
        # 使用临时文件来存储子进程输出
        with tempfile.NamedTemporaryFile(delete=False) as tmp_file:
            process = subprocess.Popen(['/Users/anelloyi/Desktop/run_powermetrics.sh'], 
                                       stdout=tmp_file, stderr=subprocess.STDOUT, text=True)
            print("Subprocess started.")
            while not stop_event.is_set():
                if process.poll() is not None:  # 检查进程是否已经结束
                    break
            
            if process.poll() is None:
                # 尝试终止进程
                process.terminate()
                try:
                    process.wait(timeout=0.1)
                except subprocess.TimeoutExpired:
                    process.kill()  # 如果超时，则强制终止
                    process.wait()
        
        # 读取临时文件的内容
        with open(tmp_file.name, 'r') as f:
            thread_output['powermetrics'] = f.read()
        
        os.remove(tmp_file.name)  # 删除临时文件
        print("Subprocess finished.")
    except Exception as e:
        thread_output['powermetrics'] = str(e)
        print("Exception in subprocess:", str(e))


stop_event = threading.Event()
model = tf.keras.models.load_model("./models_train/imdb_lstm_model.h5") 

# 创建和启动线程
thread1 = threading.Thread(target=save_model, args=(stop_event, model))
thread2 = threading.Thread(target=monitor_resources_during_save, args=(stop_event,))
thread3 = threading.Thread(target=run_script, args=(stop_event,))


thread1.start()
thread2.start()
thread3.start()

# 等待线程完成
thread1.join()
thread2.join()
thread3.join()

# 输出从线程收集的数据
content = thread_output.get('powermetrics', 'No output captured')
lines = content.split('\n')
filtered_lines = [line for line in lines if line.startswith('GPU Power:') or 
                  line.startswith('CPU Power:')]

filtered_content = '\n'.join(filtered_lines)

output_file_name = './imbd_models/output_lstm/output-imdb-lstm-pth.txt'
with open(output_file_name, 'w') as file:
    file.write(filtered_content)
    file.write(f'\nTotal Duration(s): {duration:.2f}')
    file.write(f'\nInference Duration(s): {inference_duration:.4f}')
print(f"Content saved to {output_file_name}")

filtered_lines_count = len(filtered_lines)

duration = inference_duration
# 确保 filtered_lines_count 不为零
if filtered_lines_count > 0:
    # 提取每一个采样点的数字，即CPU和GPU的具体mV
    numbers = []
    for line in filtered_lines:
        match = re.search(r'[\d.]+', line)
        if match:
            numbers.append(float(match.group()))

    delta_time = duration * 2 / filtered_lines_count
    numbers_scaled = [num * delta_time for num in numbers]
    total_energy_consumption = sum(numbers_scaled)
    print(f"Total energy consumption: {total_energy_consumption:.2f} mV")
else:
    print("No filtered lines to process.")

Saving model...
Subprocess started.
Time taken to save PyTorch model: 0.95 seconds
Time taken for inference on 10000 samples: 27.1826 seconds
Subprocess finished.
Resource monitoring finished.
Content saved to ./imbd_models/output_lstm/output-imdb-lstm-pth.txt
Total energy consumption: 193130.05 mV
